In [36]:
import pandas as pd
from pathlib import Path
from concurrent.futures import ThreadPoolExecutor, as_completed
from aspiratio.utils.report_downloader import find_annual_reports

repo = Path('..').resolve()
coverage_path = repo / 'coverage_table_updated.csv'
validation_path = repo / 'validation_results.csv'

coverage = pd.read_csv(coverage_path, sep='\t')
years = list(range(2019, 2025))

# Load validation results to identify already-validated reports
validated_df = pd.read_csv(validation_path)
validated_set = set(
    (row['CID'], row['Year']) 
    for _, row in validated_df[validated_df['Valid'] == True].iterrows()
)
print(f"✓ Already validated: {len(validated_set)} company-year combinations")

# Filter to incomplete entries (not Complete ✓)
incomplete = coverage[coverage['Priority'] != 'Complete ✓']

# Further filter out any company-year combos that are already validated
def needs_search(row):
    return (row['Company_Identifier'], row['FiscalYear']) not in validated_set

incomplete_needing_search = incomplete[incomplete.apply(needs_search, axis=1)]
print(f"→ Incomplete in coverage: {len(incomplete)}")
print(f"→ Actually need to search: {len(incomplete_needing_search)}")

incomplete_needing_search[['CompanyName', 'Company_Identifier', 'FiscalYear', 'IR_URL']].head(10)

✓ Already validated: 0 company-year combinations
→ Incomplete in coverage: 180
→ Actually need to search: 180


,CompanyName,Company_Identifier,FiscalYear,IR_URL
0,ABB Ltd,S1,2019,https://global.abb/group/en/investors
1,ABB Ltd,S1,2020,https://global.abb/group/en/investors
2,ABB Ltd,S1,2021,https://global.abb/group/en/investors
3,ABB Ltd,S1,2022,https://global.abb/group/en/investors
4,ABB Ltd,S1,2023,https://global.abb/group/en/investors
5,ABB Ltd,S1,2024,https://global.abb/group/en/investors
6,Addtech B,S2,2019,https://www.addtech.com/investors-and-media
7,Addtech B,S2,2020,https://www.addtech.com/investors-and-media
8,Addtech B,S2,2021,https://www.addtech.com/investors-and-media
9,Addtech B,S2,2022,https://www.addtech.com/investors-and-media


In [37]:
# Build search tasks: group by company and collect missing years
search_tasks = {}
for company, group in incomplete_needing_search.groupby(['CompanyName', 'Company_Identifier', 'IR_URL']):
    name, cid, ir_url = company
    target_years = sorted(group['FiscalYear'].unique().tolist())
    search_tasks[(name, cid, ir_url)] = target_years

print(f"🔍 Search tasks: {len(search_tasks)} companies")
for (name, cid, _), yrs in list(search_tasks.items())[:5]:
    print(f"   {name} ({cid}): years {yrs}")

🔍 Search tasks: 30 companies
   ABB Ltd (S1): years [2019, 2020, 2021, 2022, 2023, 2024]
   ASSA ABLOY B (S4): years [2019, 2020, 2021, 2022, 2023, 2024]
   Addtech B (S2): years [2019, 2020, 2021, 2022, 2023, 2024]
   Alfa Laval (S3): years [2019, 2020, 2021, 2022, 2023, 2024]
   AstraZeneca (S5): years [2019, 2020, 2021, 2022, 2023, 2024]


In [38]:
# Search for annual reports in parallel with Cision and MFN fallbacks
import importlib
import aspiratio.utils.report_downloader as rd_module
import aspiratio.tier1.mfn_search as mfn_module
import aspiratio.tier1.cision_search as cision_module

# Reload modules to pick up any changes
importlib.reload(mfn_module)
importlib.reload(cision_module)

from aspiratio.utils.report_downloader import find_annual_reports
from aspiratio.tier1.mfn_search import find_reports_via_mfn
from aspiratio.tier1.cision_search import find_reports_via_cision

def search_company(args):
    """Worker function to search one company with Cision and MFN fallbacks."""
    name, cid, ir_url, target_years = args
    results = []
    
    # Try standard IR search first
    try:
        candidates = find_annual_reports(ir_url, years=target_years, company_name=name)
        for c in candidates:
            results.append({
                'Company_Identifier': cid,
                'CompanyName': name,
                'Year': c.get('year'),
                'URL': c.get('url'),
                'Source_Page': c.get('source_page', ir_url),
                'Title': c.get('title', ''),
                'Source': c.get('source', 'ir')
            })
    except Exception as e:
        print(f"    ⚠ IR search error for {name}: {e}")
    
    # Check which years are still misOne thing that I'm a little bit curious about in this specific step it is uh cell number three. So when it's looking for the annual reports.sing
    found_years = {r['Year'] for r in results}
    missing_years = [y for y in target_years if y not in found_years]
    
    # Fallback 1: Try Cision News (primary source for Swedish press releases)
    if missing_years:
        print(f"    🔄 {name}: Missing years {missing_years}, trying Cision fallback...")
        try:
            cision_candidates = find_reports_via_cision(name, missing_years)
            for c in cision_candidates:
                if c.get('year') in missing_years:  # Only add if still missing
                    results.append({
                        'Company_Identifier': cid,
                        'CompanyName': name,
                        'Year': c.get('year'),
                        'URL': c.get('url'),
                        'Source_Page': c.get('source_page', ''),
                        'Title': c.get('title', ''),
                        'Source': 'cision'
                    })
                    if c.get('year') in missing_years:
                        missing_years.remove(c.get('year'))
        except Exception as e:
            print(f"    ⚠ Cision fallback error for {name}: {e}")
    
    # Fallback 2: Try MFN if still missing years
    if missing_years:
        print(f"    🔄 {name}: Still missing {missing_years}, trying MFN fallback...")
        try:
            mfn_candidates = find_reports_via_mfn(name, missing_years)
            for c in mfn_candidates:
                results.append({
                    'Company_Identifier': cid,
                    'CompanyName': name,
                    'Year': c.get('year'),
                    'URL': c.get('url'),
                    'Source_Page': c.get('source_page', ''),
                    'Title': c.get('title', ''),
                    'Source': 'mfn'
                })
        except Exception as e:
            print(f"    ⚠ MFN fallback error for {name}: {e}")
    
    status = 'success' if results else 'no_results'
    return name, cid, status, results

# Prepare tasks
task_list = [
    (name, cid, ir_url, yrs) 
    for (name, cid, ir_url), yrs in search_tasks.items()
]

# Run searches in parallel (max 5 concurrent to avoid overwhelming servers)
report_candidates = []
max_workers = min(5, len(task_list)) if task_list else 1

print(f"🚀 Starting parallel search with {max_workers} workers...")
print(f"   Companies to search: {len(task_list)}")
print(f"   Fallback chain: IR → Cision → MFN")

with ThreadPoolExecutor(max_workers=max_workers) as executor:
    futures = {executor.submit(search_company, task): task for task in task_list}
    
    for future in as_completed(futures):
        name, cid, status, results = future.result()
        if results:
            print(f"  ✓ {name} ({cid}): found {len(results)} candidates")
        else:
            print(f"  ✗ {name} ({cid}): no candidates found")
        report_candidates.extend(results)

print(f"\n📊 Total candidates found: {len(report_candidates)}")
candidates_df = pd.DataFrame(report_candidates)
candidates_df.head(10)

🚀 Starting parallel search with 5 workers...
   Companies to search: 30
   Fallback chain: IR → Cision → MFN
  Detected ABB - will search for Group Annual Reports (excluding SEC filings)...
Searching: https://global.abb/group/en/investors
  Detected ASSA ABLOY - trying direct PDF pattern...
Searching: https://www.addtech.com/investors-and-media
Searching: https://www.alfalaval.com/investors/
Searching: https://www.astrazeneca.com/investor-relations/contacts-information.html
  → Found 1366 links to analyze (from HTML, JSON, nav, footer)
  Searching: https://global.abb/group/en/investors/overview
  → Found 1783 links to analyze (from HTML, JSON, nav, footer)
  → Analyzed 1 PDF links, excluded 0 quarterly/SEC filings
  Searching: https://www.astrazeneca.com/investor-relations.html
  → Found 0 links to analyze (from HTML, JSON, nav, footer)
  No reports found in standard search, trying common patterns...
Searching: https://www.alfalaval.com/investors/annual-reports
    → Found 1365 links t

,Company_Identifier,CompanyName,Year,URL,Source_Page,Title,Source
0,S4,ASSA ABLOY B,2024,https://www.assaabloy.com/group/en/documents/i...,https://www.assaabloy.com/group/en/investors,Annual Report 2024,ir
1,S4,ASSA ABLOY B,2023,https://www.assaabloy.com/group/en/documents/i...,https://www.assaabloy.com/group/en/investors,Annual Report 2023,ir
2,S4,ASSA ABLOY B,2022,https://www.assaabloy.com/group/en/documents/i...,https://www.assaabloy.com/group/en/investors,Annual Report 2022,ir
3,S4,ASSA ABLOY B,2021,https://www.assaabloy.com/group/en/documents/i...,https://www.assaabloy.com/group/en/investors,Annual Report 2021,ir
4,S4,ASSA ABLOY B,2020,https://www.assaabloy.com/group/en/documents/i...,https://www.assaabloy.com/group/en/investors,Annual Report 2020,ir
5,S4,ASSA ABLOY B,2019,https://www.assaabloy.com/group/en/documents/i...,https://www.assaabloy.com/group/en/investors,Annual Report 2019,ir
6,S2,Addtech B,2024,https://www.addtech.com/fileadmin/user_upload/...,https://www.addtech.com/investors-and-media/an...,pdf,ir
7,S2,Addtech B,2023,https://www.addtech.com/fileadmin/user_upload/...,https://www.addtech.com/investors-and-media/an...,pdf,ir
8,S2,Addtech B,2022,https://www.addtech.com/fileadmin/user_upload/...,https://www.addtech.com/investors-and-media/an...,pdf,ir
9,S2,Addtech B,2021,https://www.addtech.com/fileadmin/user_upload/...,https://www.addtech.com/investors-and-media/an...,pdf,ir


In [39]:
# Persist candidates for review (only if we found any)
if not candidates_df.empty:
    output = repo / 'report_candidates.csv'
    candidates_df.to_csv(output, index=False)
    print(f"✓ Saved {len(candidates_df)} candidates to {output}")
else:
    print("⚠ No candidates found - nothing to save")

✓ Saved 176 candidates to /Users/jakobjohannesson/Documents/github_repos/aspiratio/report_candidates.csv
